In [ ]:
pip install tkvideo

In [2]:
from tkinter import *
from PIL import ImageTk, Image
import time




import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
from tkinter.messagebox import showinfo
from tkvideo import tkvideo

import cv2
import sys
import numpy as np
import glob
import matplotlib.pyplot as plt
%matplotlib inline

global size

import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
from tkinter.messagebox import showinfo
global filename
# create the root window
root = tk.Tk()
root.title('Stabbing action detection')
root.geometry('700x350')
root.resizable(width=True, height=True)
app = Frame(root, bg="white")
label =Label(root)
label.pack()
Label(root, text="Stabbing action detection using State-of-the Art",font=("Arial", 25) ).pack()
# Create a label in the frame

start_time=time.time()



def select_file():
    filename=fd.askopenfilename(filetypes=(("Video files", "*.mp4"),))

    showinfo(
        title='Selected File',
        message=filename
        
    )
    return filename

def gmm_run():
    x=select_file()
  
    video = cv2.VideoCapture(x)

# You can set custom kernel size if you want.
    kernel = None
#mask = 0
# Initialize the background object.
    backgroundObject = cv2.createBackgroundSubtractorMOG2(detectShadows = True)
    n=0
#now = datetime.now()
#current_time = now.strftime("%H:%M:%S")
#print("Start Time =", current_time)
    while True:
   
    # Read a new frame.
        ret, frame = video.read()
  #  status = cv2.imwrite('/F:/CODES/FOREGROUNDDETECTION/1.png',frame)
    # Check if frame is not read correctly.
        if not ret:
       
        # Break the loop.

            break

    # Apply the background object on the frame to get the segmented mask.
        fgmask = backgroundObject.apply(frame)
    #initialMask = fgmask.copy()
   
   
    # Perform thresholding to get rid of the shadows.
        _, fgmask = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)
    #noisymask = fgmask.copy()
   
    # Apply some morphological operations to make sure you have a good mask
        fgmask = cv2.dilate(fgmask, kernel, iterations = 2)
        fgmask = cv2.erode(fgmask, kernel, iterations = 1)

   
   
    # define the kernel
        kernel = np.ones((3, 3), np.uint8)
 
    # invert the image
        invert = cv2.bitwise_not(fgmask)
 
    # use morph gradient
        fgmask = cv2.morphologyEx(invert, cv2.MORPH_GRADIENT, kernel)
    #fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel, iterations=1)
        fgmask = cv2.dilate(fgmask, kernel, iterations = 1)
   
   
    #fgmask = cv2.dilate(fgmask, kernel, iterations = 1)
    # Detect contours in the frame.
        contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
   
    # Create a copy of the frame to draw bounding boxes around the detected cars.
        frameCopy = frame.copy()
   
    # loop over each contour found in the frame.
        for cnt in contours:
       
        # Make sure the contour area is somewhat higher than some threshold to make sure its a car and not some noise.
            if cv2.contourArea(cnt) > 5000:
           
            # Retrieve the bounding box coordinates from the contour.
                x, y, width, height = cv2.boundingRect(cnt)
           # crop_img = frameCopy[x:width, y:height]

            # Draw a bounding box around the car.
                cv2.rectangle(frameCopy, (x , y), (x + width, y + height),(0, 0, 255), 2)
           
            # Write Car Detected near the bounding box drawn.
                cv2.putText(frameCopy, 'Person Detected', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0), 1, cv2.LINE_AA)
   
    # Extract the foreground from the frame using the segmented mask.
        foregroundPart = cv2.bitwise_and(frame, frame, mask=fgmask)
        stacked = np.hstack((frame, foregroundPart, frameCopy))
    #cv2.imwrite('IMAGE%04i.jpg'%n,foregroundPart)
    #n=n+1;
            #cv2.imwrite('IMAGE%04i.jpg'%n,crop_img)
            #n=n+1;
    # Stack the original frame, extracted foreground, and annotated frame.
    #stacked = np.hstack((frame, frameCopy, mask))

    # Display the stacked image with an appropriate title.
        #cv2.imshow('Original Frame, Extracted Foreground and Detected Cars', cv2.resize(stacked, None, fx=1.5, fy=1.5))
    #cv2.imshow('initial Mask', initialMask)
    #cv2.imshow('Noisy Mask', noisymask)
    #cv2.imshow('Clean Mask', fgmask)
        x, y, width, height = cv2.boundingRect(cnt)
        mask1 = np.zeros(frameCopy.shape,np.uint8)
        mask1[y:y+height,x:x+width] = frameCopy[y:y+height,x:x+width]
        stacked = np.hstack((frame, foregroundPart, frameCopy, mask1))
        cv2.imwrite('frames/IMAGE%04i.jpg'%n,mask1)
        
        #img = ImageTk.PhotoImage(Image.open(frameCopy))
        #panel = Label(root, image = img)
        #panel.pack()
        
    #path = 'D:/OpenCV/Scripts/Images'
    #path = 'F:\CODES\GMM UPDATED\GMM WITH BOUNDING BOX ONLY AFTER ADDING FEW MORPHOLOGIES'
    #cv2.imwrite(os.path.join(path , 'IMAGE%04i.jpg'%n), img)
        n=n+1;

    # Wait until a key is pressed.
    # Retreive the ASCII code of the key pressed
        k = cv2.waitKey(1) & 0xff
   
    # Check if 'q' key is pressed.
        if k == ord('q'):
       
        # Break the loop.
            break
       
    #now = datetime.now()
#current_time = now.strftime("%H:%M:%S")

#print("End Time =", current_time)

# Release the VideoCapture Object.
    video.release()

# Close the windows.q
    cv2.destroyAllWindows()


    img_array = []
    for filename in glob.glob('frames/*.jpg'):
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)


    out = cv2.VideoWriter('yolov5/GOA_GMM.avi',cv2.VideoWriter_fourcc(*'DIVX'), 35, size)

#cv2.VideoWriter("/your/path/filename.mp4",fourcc, 20,(320,180))

 
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()
    
    player = tkvideo("yolov5/GOA_GMM.avi", label, loop = 2, size = (640,360))
    player.play()
"""
cap= cv2.VideoCapture('GOA_GMM.avi')

# Define function to show frame
def show_frames():
   # Get the latest frame and convert into Image
   cv2image= cv2.cvtColor(cap.read()[1],cv2.COLOR_BGR2RGB)
   img = Image.fromarray(cv2image)
   # Convert image to PhotoImage
   imgtk = ImageTk.PhotoImage(image = img)
   label.imgtk = imgtk
   label.configure(image=imgtk)
   # Repeat after an interval to capture continiously
   label.after(20, show_frames)
"""
"""
# open button
open_button = ttk.Button(
    root,
    text='Upload cctv footage',
    command=select_file
)

open_button.pack(expand=True)

"""


def yolo_run():
    
    %run YOLOv5OnPC.ipynb

def opticalflow_run():
    
    
    
    %run opticalflowoutput.ipynb
    
    

gmm_button=Button(root,text='Foreground Segmentation',command=gmm_run).pack()


yolo_button=Button(root,text='YoloV5',command=yolo_run).pack()
opticalflow_button=Button(root,text='Optical Flow',command=opticalflow_run).pack()
#gmm_button.pack(expand=True)
# run the application

root.mainloop()
end_time=time.time()
print(end_time-start_time)


6.639337539672852
